Задание
    1. на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
    2. Проверить насколько хорошо работают подходы

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import annoy
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText
from tqdm.notebook import tqdm

exclude = set(string.punctuation)

In [2]:
HOME_DIR = Path('..')
INPUT_DIR = HOME_DIR / 'input/l3/'

train_df = pd.read_csv(INPUT_DIR / 'lenta-ru-news.csv')
train_df['title'] = train_df['title'].str.replace('\xa0', ' ')
train_df.head()

C:\Users\Alexidis\AppData\Local\Temp\ipykernel_17112\1907254647.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(INPUT_DIR / 'lenta-ru-news.csv')


,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [3]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [l for i in spls if (l:=morpher.parse(i.lower())[0].normal_form) not in sw and l != ""]
    return spls

In [4]:
def get_response(q, index, model, i_map):
    q = preprocess_txt(q)
    vector = np.zeros(300)
    a = [vector for word in q if word in model.wv]
    if (norm:= len(a)):
        vector = sum(a) / norm

    answers = index.get_nns_by_vector(vector, 5, )
    return [i_map[i] for i in answers]

In [5]:
# Preprocess for models fitting

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))

In [6]:
max_row_count = 10000

sentences = [preprocess_txt(text) for text in  tqdm(train_df['text'][:max_row_count])]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
%%time
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

CPU times: total: 23.1 s
Wall time: 8.85 s


In [8]:
%%time
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

CPU times: total: 8min 8s
Wall time: 1min 21s


In [9]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in tqdm(train_df[['text', 'title']][:max_row_count].values):
    n_w2v = 0
    n_ft = 0
    # spls = line.split("\t")
    index_map[counter] = line[1]
    text = preprocess_txt(line[0])

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in text:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
w2v_index.build(10)
ft_index.build(10)

True

In [11]:
TEXT = "Политика"

In [12]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Правительство Чечни считает, что Россия начала войну',
 'Еще 11 заложников вернулись из чеченского плена',
 'Рушайло и Квашнин вылетели на Кавказ',
 'Территория Дагестана полностью очищена от боевиков',
 'Границы и воздушное пространство Чечни закрыты']

In [13]:
get_response(TEXT, ft_index, modelFT, index_map)

['Создан департамент ФСБ по борьбе с терроризмом',
 'Американские миротворцы скрывали факт массового убийства',
 'Спецпредставитель президента РФ Михаил Задорнов уходит со своего поста',
 'Армия освобождения Косово не будет расформирована',
 'Россия недовольна процессами в Косово']